In [1]:
from tdc.generation import MolGen
from tdc.single_pred import ADME
from tdc.single_pred import Tox
from tdc.utils import retrieve_label_name_list
import pandas as pd

path = "/home/bhux/workplace/drug_discovery/data"

def get_splits(data): 
    split = data.get_split()
    split = [split['train'], split['valid'], split['test']]
    split = pd.concat(split)
    return split

adme_names = [
    'Caco2_Wang',   # 906
    'Lipophilicity_AstraZeneca',
    'Solubility_AqSolDB',
    'HydrationFreeEnergy_FreeSolv',
    'PPBR_AZ',
    'VDss_Lombardo',
    'Half_Life_Obach',
    'Clearance_Hepatocyte_AZ',
    'Clearance_Microsome_AZ',
]

tox_names = [
    'LD50_Zhu',
]

data_dfs = {}
for name in adme_names:
    data = ADME(name = name, path=path)
    data_dfs[name] = get_splits(data)

data = Tox(name = 'LD50_Zhu', path=path)
data_dfs['LD50_Zhu'] = get_splits(data)

label_list = retrieve_label_name_list('herg_central')
for lname in label_list[:-1]: # no inhib
    data = Tox(name = 'herg_central', label_name = lname)
    data_dfs['herg_central_'+lname] = get_splits(data)

print(data_dfs.keys())

# categorical adme 
cat_adme_names = [
    'PAMPA_NCATS',
    'HIA_Hou',
    'Pgp_Broccatelli',
    'Bioavailability_Ma',
    'BBB_Martins',
    # 'CYP2C19_Veith',
    # 'CYP2D6_Veith',
    # 'CYP3A4_Veith',
    # 'CYP1A2_Veith',
    # 'CYP2C9_Veith',
    # 'CYP2C9_Substrate_CarbonMangels',
    # 'CYP2D6_Substrate_CarbonMangels',
    # 'CYP3A4_Substrate_CarbonMangels',
]

for name in cat_adme_names:
    data = ADME(name = name, path=path)
    data_dfs[name] = get_splits(data)

# cat_tox_names = [
#     'AMES',
#     'DILI',
#     'Skin Reaction',
#     'Carcinogens_Lagunin',
#     'ClinTox',
# ]

# for name in cat_tox_names:
#     data = Tox(name = name, path=path)
#     data_dfs[name] = get_splits(data)

# # Tox21 and ToxCast
# name = 'Tox21'
# label_list = retrieve_label_name_list(name)
# for lname in label_list: # no inhib
#     data = Tox(name = name, label_name = lname)
#     data_dfs[name+lname] = get_splits(data)

# name = 'Toxcast'
# label_list = retrieve_label_name_list(name)
# for lname in label_list[:10]: # no inhib
#     data = Tox(name = name, label_name = lname)
#     data_dfs[name+lname] = get_splits(data)

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


dict_keys(['Caco2_Wang', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB', 'HydrationFreeEnergy_FreeSolv', 'PPBR_AZ', 'VDss_Lombardo', 'Half_Life_Obach', 'Clearance_Hepatocyte_AZ', 'Clearance_Microsome_AZ', 'LD50_Zhu', 'herg_central_hERG_at_1uM', 'herg_central_hERG_at_10uM'])


In [2]:
import numpy as np

# Populate smiles
smiles = []
for _, df in data_dfs.items():
    smiles.extend(list(df['Drug']))
smiles = list(set(smiles))
print(len(smiles))

# Build dataset
lengths = []
dataset = {'Drug': smiles}
for k, df in data_dfs.items():
    df = {v['Drug']: v['Y'] for v in df.to_dict(orient='records')}
    dataset[k] = []
    for smile in smiles:
        dataset[k].append(df.get(smile, np.nan))
    lengths.append(len(dataset[k])-sum(np.isnan(np.array(dataset[k]))))

print(dataset.keys())

df = pd.DataFrame(data=dataset)

display(df[:10])



332107
dict_keys(['Drug', 'Caco2_Wang', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB', 'HydrationFreeEnergy_FreeSolv', 'PPBR_AZ', 'VDss_Lombardo', 'Half_Life_Obach', 'Clearance_Hepatocyte_AZ', 'Clearance_Microsome_AZ', 'LD50_Zhu', 'herg_central_hERG_at_1uM', 'herg_central_hERG_at_10uM', 'PAMPA_NCATS', 'HIA_Hou', 'Pgp_Broccatelli', 'Bioavailability_Ma', 'BBB_Martins'])


,Drug,Caco2_Wang,Lipophilicity_AstraZeneca,Solubility_AqSolDB,HydrationFreeEnergy_FreeSolv,PPBR_AZ,VDss_Lombardo,Half_Life_Obach,Clearance_Hepatocyte_AZ,Clearance_Microsome_AZ,LD50_Zhu,herg_central_hERG_at_1uM,herg_central_hERG_at_10uM,PAMPA_NCATS,HIA_Hou,Pgp_Broccatelli,Bioavailability_Ma,BBB_Martins
0,CCn1c(SCC(=O)N2CCOCC2)nc(O)cc1=O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.1332,14.23458,NaN,NaN,NaN,NaN,NaN
1,Cc1cccc(OCc2nc(-c3ccc(-n4cccc4)cc3)no2)c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.1856,12.18548,NaN,NaN,NaN,NaN,NaN
2,COc1ccc(-c2csc(-c3ccccc3)n2)c(OC)c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.7342,29.07057,NaN,NaN,NaN,NaN,NaN
3,O=C(O)c1nn(Cc2nc3ccccc3o2)c(=O)c2ccccc12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.2438,20.56388,NaN,NaN,NaN,NaN,NaN
4,Cc1ccccc1C(=O)NC(CC(C)C)C(=O)NN1C(=O)c2ccccc2C1=O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.5830,25.92138,NaN,NaN,NaN,NaN,NaN
5,CC1CCc2ccccc2N1C(N)=Nc1ccccc1.Cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5059,5.34678,NaN,NaN,NaN,NaN,NaN
6,CN1[C@H]2CCC[C@@H]1CC(NC(=O)c1nn(C)c3ccccc13)C2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,CCCCCCc1ccc(N2C[C@H](Cc3ccccc3)[C@@H](CC(=O)NC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6149,-4.41902,NaN,NaN,NaN,NaN,NaN
8,COc1cccc(CNC(=O)CCS(=O)(=O)c2cccc3nonc23)c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0490,11.17348,NaN,NaN,NaN,NaN,NaN
9,Cc1ccc(-c2[nH]c3ccccc3c2C(C[N+](=O)[O-])c2ccco...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.6262,8.51158,NaN,NaN,NaN,NaN,NaN


In [3]:
from collections import Counter

combinations = []
for row in df.drop(labels=['Drug', 'herg_central_hERG_at_10uM'], axis=1).iterrows():
    combinations.append("".join([str(int(np.isnan(v) == False)) for _,v in list(row)[1].items()]))

uc = Counter(combinations)
print(uc.keys())
print(uc.values())

dict_keys(['0000000000100000', '0000000000000001', '1000000000000000', '0000000001000000', '0110000001100000', '0100000000000000', '0010000000000000', '0000010000000000', '0000000000010000', '0100000000100000', '0000000100000000', '0100100110000000', '0100100000000000', '0000000001100000', '0000000000001000', '0000100000100000', '0010000001000000', '0000000000000100', '0000100110000000', '0010000000100000', '0000011001101111', '0100000010000000', '0010000000000010', '0110001000100001', '0000000000000010', '0100100010000000', '0100000110000000', '0011000001000000', '0010000001100000', '0000000001000001', '0100100010000011', '0011000000000000', '0000001000100000', '0000000110000000', '0000001000000011', '0000000000100100', '1000001000000010', '0000100000000000', '0100000000000100', '0001000000000000', '0000011000101010', '1010000000000001', '0010000000000100', '0000000000101111', '0000001000000000', '0100000000001000', '0100100000100110', '0011000001000001', '0000010000100000', '01100000

In [4]:
cbool = np.array(combinations) != '0'*10 + '1' + '0'*(len(list(uc.keys())[0])-11) #'00000000001000000000000000000000000000000'
inc = np.random.choice([i for i, x in enumerate(cbool) if x == False], 7900, replace=False)
cbool = [True if i in inc else v for i,v in enumerate(cbool)]

fcomb = np.array(combinations)[cbool]
fdf = df[cbool]

print(len(fdf.index))

34725


In [5]:
def unison_shuffled_copies(a, b):
    #assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], p

rcomb, perm = unison_shuffled_copies(fcomb, fdf.iloc)
lengths[10] = 7900
del lengths[11]
valCount = np.array(lengths)*0.1

trIdx = []
valIdx = []
for c, p in zip(rcomb, perm):
    inc = False
    for i, j in enumerate(list(c)):
        if j == "1" and valCount[i] > 0:
            valCount[i] -= 1
            inc = True
    if inc:
        valIdx.append(p)
    else:
        trIdx.append(p)

print(len(trIdx))
print(len(valIdx))

31301
3424


In [6]:
trdf = fdf.iloc[trIdx]
valdf = fdf.iloc[valIdx]

In [7]:
from transformers import AutoTokenizer, AutoModel, T5EncoderModel, AutoModelForMaskedLM
import torch
from tqdm import tqdm

model = "sagawa/PubChem-10m-deberta"
# "sagawa/PubChem-10m-deberta"
# "seyonec/ChemBERTa-zinc-base-v1"
# "DeepChem/ChemBERTa-10M-MLM"
# "sagawa/PubChem-10m-t5-v2"

# "ncfrey/ChemGPT-4.7M"
# "RaphaelMourad/Mistral-Chem-v1-417M"
# "jonghyunlee/ChemBERT_ChEMBL_pretrained"

# Initialize tokenizer and model for drug embeddings
drug_tokenizer = AutoTokenizer.from_pretrained(model)
drug_model = AutoModel.from_pretrained(model)

drug_model.to('cuda')

# Function to get drug embeddings from SMILES strings
def get_drug_embeddings(smiles_list, tokenizer, model, batch_size=64):
    embeddings = []
    for i in tqdm(range(0, len(smiles_list), batch_size)):
        batch = smiles_list[i:i+batch_size]
        encoding = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        input_ids = encoding["input_ids"].to('cuda')
        attention_mask = encoding["attention_mask"].to('cuda')  # Get the attention mask
        
        with torch.no_grad():
            # Pass both input_ids and attention_mask to the model
            output = model(input_ids, attention_mask=attention_mask)
            # You can use either the last hidden state or the pooled output
            # depending on your requirements. Here we're using the mean of the last hidden state.
            embedding = output.last_hidden_state.mean(1)
            embeddings.extend(embedding.cpu().numpy())
            
    return embeddings

In [8]:
# Save train
drug_embeddings_split_kd = get_drug_embeddings(list(fdf['Drug']), drug_tokenizer, drug_model)
print(len(drug_embeddings_split_kd))
np.save("./data/xtended_emb_all_deberta_pubchem", np.array(drug_embeddings_split_kd))
fdf.to_csv("./data/xtended_data_all.csv", index=False)

100%|██████████| 543/543 [00:53<00:00, 10.14it/s]


34725


In [9]:
drug_embeddings_split_kd = get_drug_embeddings(list(valdf['Drug']), drug_tokenizer, drug_model)
print(len(drug_embeddings_split_kd))
np.save("./data/xtended_emb_val_deberta_pubchem", np.array(drug_embeddings_split_kd))
valdf.to_csv("./data/xtended_data_val.csv", index=False)

100%|██████████| 54/54 [00:05<00:00, 10.62it/s]

3424
